In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate
import scipy.sparse as sp

In [ ]:
#constants
hbar, m = 1, 1
k0, L = 20, 20
Nx = 1000 # number of x-values
dx = L / (Nx - 1)
sigmax = 1
x0 = 5

#Find values 
dt = 0.01
E = (hbar * k0) ** 2 / 2 / m
omega = E / hbar


def psi_I(x, t):
    return np.exp(-(x - x0)**2 / (2 * sigmax**2)) * np.sin(k0 * x - omega * t)
                  
def psi_R(x,t):
    return np.exp(-(x - x0)**2 / (2 * sigmax **2)) * np.cos(k0 * x - omega * t)

x = np.linspace(0, L, Nx)
psi_I_init = psi_I(x, 0)
psi_R_init = psi_R(x, 0)

C_0 = np.trapz(psi_I(x, 0)**2 + psi_R(x, 0)**2, dx = dx)
psi_R_init = psi_R(x, 0) / np.sqrt(C_0)

C_1 = np.trapz(psi_R(x, dt/ 2)**2 + psi_R(x, dt / 2)**2, dx = dx)
psi_I_init = psi_R(x, dt/2) / np.sqrt(C_1)

print(C_0, C_1)

In [ ]:
def timeEvolve(psi_R_init,psi_I_init,n,V,dt):
    x_size = len(V)
    
    psi_I_arr = np.zeros([x_size,n])
    psi_R_arr = np.zeros([x_sixe,n])
    psi_I_arr[0] = psi_I_init
    psi_R_arr[0] = psi_R_init
    
    D1 = 2 * np.ones(x_size)
    D2 = np.ones(x_size-1)
    H = np.diag(V) / hbar - hbar / (2 * m * dx**2) * sp.diags([D2,D1,D2],[-1,0,1]).toarray()
    for i in range(1,n):
        psi_R_arr[i] = psi_R_arr[i-1] + dt * H @ psi_I_arr[i-1]
        psi_I_arr[i] = psi_I_arr[i-1] - dt * H @ psi_R_arr[i-1]
    return psi_R_arr,psi_I_arr

        